<a href="https://colab.research.google.com/github/PepiMartin28/ETL-Project/blob/main/Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#In this notebook we are going to extract the data from the page "https://fbref.com/". We will work with the first five European leagues from the 2000-2001 season onwards,

###First import the necesaries modules

In [ ]:
import pandas as pd
import csv
import time

###Lets create the csv file where the data is stored for now.

In [ ]:
columns_csv = ['league_id', 'league_name', 'season', 'id', 'name', 'matches_played', 'wins', 'draws', 'losses', 'season_points',
               'goals_scored', 'shots_on_target', 'goals_against', 'shots_against', 'goal_diff', 'clean_sheets', 'yellow_cards', 'red_cards']

with open('/content/drive/MyDrive/Proyectos/ETL/data.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(columns_csv)

The next command is the process to extract the data. We use the "time.sleep(10)" function to stop the process for 10 seconds to avoid being blocked from the page for 1 hour.

We are using the next columns:

* league_id
* league_name
* season
* id
* name (team name)
* matches_played (matches played in this season)
* wins (matches won in this season)
* draws (matches drawn in this season)
* losses (matches lost in this season)
* season_points (points obtained in this season)
* goals_scored (goals scored in this season)
* shots_on_target (shots made in this season)
* goals_against (goals received in this season)
* shots_against (shots received in this season)
* goal_diff (difference between goals_scored and goals_against)
* clean_sheets (clean sheets made in this season)
* yellow_cards (yellow cards received this season)
* red_cards (red cards received this season)

In [ ]:
leagues = ['Premier-League', 'La-Liga', 'Serie-A', 'Ligue-1', 'Bundesliga']
leagues_id = [9, 12, 11, 13, 20]
id_counter = 0
loaded_teams = {}
rows = []

for idx in range(5):
  for season in range(2000, 2023):
    try:
      querie = pd.read_html(f'https://fbref.com/en/comps/{leagues_id[idx]}/{season}-{season+1}/Estadisticas-{season}-{season+1}-{leagues[idx]}')
    except:
      continue

    df_season = querie[0]
    df_stats = querie[2]
    df_stats.columns = df_stats.columns.droplevel(0)
    df_additional_stats = querie[4]
    df_additional_stats.columns = df_additional_stats.columns.droplevel(0)
    df_shots_stats = querie[6]
    if season >= 2017:
      df_shots_stats = querie[8]
    df_shots_stats.columns = df_shots_stats.columns.droplevel(0)

    teams = [team for team in df_season['Squad'].values]

    for team in teams:
      df_team_season = df_season[df_season['Squad'] == team]
      df_team_stats = df_stats[df_stats['Squad'] == team]
      df_team_additional_stats = df_additional_stats[df_additional_stats['Squad'] == team]
      df_team_shots_stats = df_shots_stats[df_shots_stats['Squad'] == team]

      try:
        team_id = loaded_teams[team]
      except:
        loaded_teams[team] = id_counter
        team_id = id_counter

      rows.append([leagues_id[idx], leagues[idx].replace("-", " "), f'{season}-{season+1}', team_id, team,
                   df_team_season['MP'].values[0], df_team_season['W'].values[0], df_team_season['D'].values[0], df_team_season['L'].values[0],
                   df_team_season['Pts'].values[0], df_team_season['GF'].values[0], df_team_shots_stats['SoT'].values[0],
                   df_team_season['GA'].values[0], df_team_additional_stats['SoTA'].values[0],
                   df_team_season['GF'].values[0] - df_team_season['GA'].values[0],
                   df_team_additional_stats['CS'].values[0], df_team_stats['CrdY'].values[0], df_team_stats['CrdR'].values[0]])

      id_counter += 1

    time.sleep(5)

with open('/content/drive/MyDrive/Proyectos/ETL/data.csv', 'a') as csv_file:
  writer = csv.writer(csv_file)
  for row in rows:
    writer.writerow(row)